In [31]:
import pyomo.environ as pyomo

# solver
solver = pyomo.SolverFactory('cbc',executable=r'C:\Cbc-master-win64-msvc16-mt\bin\cbc.exe')
solver.options['sec'] = 5

In [46]:
import pandas as pd 
sdf = pd.read_csv('seller_pyomo_data_single_t_period.csv')
bdf = pd.read_csv('buyer_pyomo_data_single_t_period.csv')

In [47]:
print(sdf)

   C_geninv_it  C_gen_it  f_it  CO2_it   D_it  PS_it
0         0.01      0.01  0.01   10.00      5    1.0
1         0.01      0.01  0.01    0.01      5    1.0
2         0.01      0.01  0.01   10.00      0    1.0
3         0.50      0.50  0.01   10.00      5    1.0
4         1.00      2.00  0.01   10.00      5    1.0
5         0.01      0.01  0.01    0.01      8    0.5
6         0.01      0.01  0.01   10.00  10000    1.0


In [48]:
print(bdf)

   C_geninv_it  C_gen_it  f_it  CO2_it   D_it  PS_it
0         0.01      0.01  0.01   10.00      5    1.0
1         0.01      0.01  0.01    0.01      5    1.0
2         0.01      0.01  0.01   10.00      0    1.0
3         0.50      0.50  0.01   10.00      5    1.0
4         1.00      2.00  0.01   10.00      5    1.0
5         0.01      0.01  0.01    0.01      8    0.5
6         0.01      0.01  0.01   10.00  10000    1.0


buyer 

In [35]:
# constraints 

#sum_{i,t}Y_{i,t}<= Q^F_i 
def buyer_request(model):
   return model.Y_it <= model.QF_i 
#model.requestConstraint = pyomo.Constraint(rule=request)

#  f_{i,t}*g_{i,t} <= CO2_{i,t}
def buyer_carbon_target(model):  
    return model.f_it * model.g_it <= model.CO2_it 
#model.carbontargetConstraint = pyomo.Constraint(rule=carbon_target)

# Y_{i,t} + g_{i,t} + Q^S_{i,t} = D_{i,t} 
def buyer_demand(model): 
    return (model.Y_it + model.g_it + model.QS_it == model.D_it)  
# model.demandConstraint = pyomo.Constraint(rule=demand)

#  g_{i,t} <= \sum^t_{t'=1} h_{i,t'}
def buyer_generator_capacity (model):  
    return model.g_it <= model.h_it 
#model.generatorConstraint = pyomo.Constraint(rule=generator_capacity)

lambda_dict = dict()

def buyer_model (df): 
    for i in range (len(bdf)): 
        # create a model 
        model = pyomo.ConcreteModel()

        # Parameters
        model.C_geninv_it = pyomo.Param(initialize=bdf['C_geninv_it'][i])  # C^{geninv}_{i,t}
        model.C_gen_it = pyomo.Param(initialize=bdf['C_gen_it'][i])        # C^{gen}_{i,t}
        model.f_it = pyomo.Param(initialize=bdf['f_it'][i])                # f_{i,t}
        model.CO2_it = pyomo.Param(initialize = bdf['CO2_it'][i])        #CO2_{i,t}
        model.D_it = pyomo.Param(initialize = bdf['D_it'][i])     #D_{i,t}
        model.PS_it = pyomo.Param(initialize = bdf['PS_it'][i])    # P^S_{i,t}
        model.PF_i = pyomo.Param(initialize=1.0)  
        # Variables
        model.g_it = pyomo.Var(domain=pyomo.NonNegativeReals)       # g_{i,t}
        model.h_it = pyomo.Var(domain=pyomo.NonNegativeReals)       # h_{i,t}
        model.QS_it = pyomo.Var(domain=pyomo.NonNegativeReals)      # Q^S_{i,t}
        model.QF_i = pyomo.Var(domain=pyomo.NonNegativeReals)       # Q^F_i
        model.Y_it = pyomo.Var(domain=pyomo.NonNegativeReals)     #Y_{i,t}

        # Objective Function
        #min P^F_i * Q^F_i + C^{geninv}_{i,t} * h_{i,t} + C^{gen}_{i,t} * g_{i,t} + P^S_{i,t} * Q^S_{i,t}
        def objective_func(model):
            return model.PF_i * model.QF_i \
                    + model.C_geninv_it * model.h_it \
                    + model.C_gen_it * model.g_it \
                    + model.PS_it * model.QS_it 
        model.objective = pyomo.Objective(rule = objective_func, sense = pyomo.minimize)
        # constraints 
        model.requestConstraint = pyomo.Constraint(rule=buyer_request)
        model.carbontargetConstraint = pyomo.Constraint(rule=buyer_carbon_target)
        model.demandConstraint = pyomo.Constraint(rule=buyer_demand)
        model.generatorConstraint = pyomo.Constraint(rule=buyer_generator_capacity)

        # solve 
        #dual 
        model.dual = pyomo.Suffix(direction=pyomo.Suffix.IMPORT)
        # objective 
        print(model.objective)
        result = solver.solve(model)
        print(result.solver.status)
        print(result.solver.termination_condition)
        print(model.display())

        print("check KKT conditions")
        all_ok = True 
        lambda_list = []
                
        for constr in model.component_objects(pyomo.Constraint, active=True):
            ok = True
            for idx in constr:
                c = constr[idx]
                val = pyomo.value(c.body)
                lb, ub = c.lower, c.upper

                # Get dual variable value 
                λ = model.dual.get(c, None)

                # primal 
                print("primal")
                if lb is not None and val < lb :
                    print(f"[Primal Infeasibility] {c.name}: {val} < {lb}")
                    all_ok = False
                elif ub is not None and val > ub :
                    print(f"[Primal Infeasibility] {c.name}: {val} > {ub}")
                    all_ok = False
                
                # ---- dual ----
                print('dual')
                if λ is not None:
                    if lb is None and ub is not None:  # ≤ constraint # neg 
                        if λ > 0:
                            print(f"[Dual Infeasibility] {c.name}: λ = {λ} > 0")
                            all_ok = False
                    elif lb is not None and ub is None:  # ≥ constraint #non-neg 
                        if λ < 0:
                            print(f"[Dual Infeasibility] {c.name}: λ = {λ} < 0")
                            all_ok = False
                    elif lb == ub:  # equality constraint — no sign restriction
                        if lb == 0 and λ != 0: # when D = 0 
                            print (f"[Dual Infeasibility] {c.name}: λ = {λ}" ) 
                        elif lb > 0 and λ <= 0 : # when D > 0 
                            print(f"[Dual Infeasibility] {c.name}: λ = {λ} - no sign restriction")
                        pass
            if ok:     
                lambda_list.append (λ) 
        λ_1 = lambda_list[0]
        λ_2 = lambda_list[1]
        λ_3 = lambda_list[2]
        λ_4 = lambda_list[3]    

        #dual variable 
        f = model.CO2_it*λ_2 + model.D_it*λ_3
        print ("dual objective function", f)

        print("Complementary slackness")
        #constraints 27-29 
        if (λ_1 < 0) or (float(pyomo.value(model.QF_i - model.Y_it)) < 0) : 
            all_ok = False
        if (λ_2 < 0) or (float(pyomo.value(model.CO2_it - model.f_it * model.g_it)) < 0): 
            all_ok = False 
        if (λ_4 < 0) or (float(pyomo.value(model.h_it - model.g_it)) < 0): 
            all_ok = False 
        
        if all_ok == False:
            print("Some KKT conditions failed")
        print("done with KKT checks\n")

        key = "lambda_list"+str(i)  
        lambda_dict[key] = lambda_list
        print(f"λ1, λ2, λ3, λ4 = {lambda_list}")

        print ("use stationarity constraint to derive λ_QF, λ_h, λ_g, λ_QS, λ_Y")
        #constraints 30-34 
        λ_QF = float(model.PF_i - λ_1)
        λ_h = float(model.C_geninv_it - λ_4)
        λ_g = float(model.C_gen_it + λ_2*model.f_it - λ_3 + λ_4)
        λ_QS = float(model.PS_it - λ_3)
        λ_Y = float(λ_1 - λ_3)
        lambda_list.append(λ_QF)
        lambda_list.append(λ_h)
        lambda_list.append(λ_g)
        lambda_list.append(λ_QS)
        lambda_list.append(λ_Y)  

        if (λ_g < 0) or (float(pyomo.value(model.g_it)) < 0 ): 
            all_ok = False
        if (λ_h < 0) or (float(pyomo.value(model.h_it)) <0): 
            all_ok = False
        if (λ_QF < 0) or (float(pyomo.value(model.QF_i)) <0): 
            all_ok = False
        if (λ_QS < 0) or (float(pyomo.value(model.QS_it)) <0): 
            all_ok = False 
        if (λ_Y < 0) or (float(pyomo.value(model.Y_it)) < 0): 
            all_ok = False 

    print(lambda_dict)       
    return model

seller 

In [44]:
buyer_model(bdf)
sum(pyomo.value(buyer_model(bdf).QF_i))

objective
ok
optimal
Model unknown

  Variables:
    g_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :  None : False : False : NonNegativeReals
    h_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :  None : False : False : NonNegativeReals
    QS_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    QF_i : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    Y_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.1

  Cons

TypeError: 'float' object is not iterable

In [ ]:
# constraints 

#sum_{i,t}Y_{i,t}<= Q^F_i 
def request(model):
   return model.Y_it <= model.QF_i 
#model.requestConstraint = pyomo.Constraint(rule=request)

#  f*g <= CO2
def carbon_target(model):  
    return model.f_t * model.g_t <= model.CO2_t
#model.carbontargetConstraint = pyomo.Constraint(rule=carbon_target)

# Y+ g+ Q^S = D
def demand(model): 
    return (model.Y_it + model.g_t + model.QS_t == model.D_it)  
# model.demandConstraint = pyomo.Constraint(rule=demand)

#  g <= h
def generator_capacity (model):  
    return model.g_t <= model.h_t 
#model.generatorConstraint = pyomo.Constraint(rule=generator_capacity)

def seller_model(df): 
    for i in range (len(sdf)): 
        # create a model 
        model = pyomo.ConcreteModel()

        # Parameters
        model.C_geninv_it = pyomo.Param(initialize=sdf['C_geninv_it'][i])  # C^{geninv}_{i,t}
        model.C_gen_it = pyomo.Param(initialize=sdf['C_gen_it'][i])        # C^{gen}_{i,t}
        model.f_t = pyomo.Param(initialize=sdf['f_it'][i])                # f_{i,t}
        model.CO2_t = pyomo.Param(initialize = sdf['CO2_it'][i])        #CO2_{i,t}
        model.D_it = pyomo.Param(initialize = sdf['D_it'][i])     #D_{i,t}
        model.PS_it = pyomo.Param(initialize = sdf['PS_it'][i])    # P^S_{i,t}
        # Variables
        model.g_t = pyomo.Var(domain=pyomo.NonNegativeReals)       # g_{i,t}
        model.h_t = pyomo.Var(domain=pyomo.NonNegativeReals)       # h_{i,t}
        model.QS_t = pyomo.Var(domain=pyomo.NonNegativeReals)      # Q^S_{i,t}
        model.QF_i = pyomo.Var(domain=pyomo.NonNegativeReals)       # Q^F_i
        model.Y_it = pyomo.Var(domain=pyomo.NonNegativeReals)     #Y_{i,t}
        model.PF_i = pyomo.Var(domain=pyomo.NonNegativeReals)       #P^F_i 

        # Objective Function
        #max P^F_i * Q^F_i - C^{geninv}_{i,t} * h_{i,t} - C^{gen}_{i,t} * g_{i,t} - P^S_{i,t} * Q^S_{i,t}
        def objective_func(model):
            return model.PF_i * model.QF_i \
                    - model.C_geninv_it * model.h_t \
                    - model.C_gen_it * model.g_t \
                    - model.PS_it * model.QS_t 
        model.objective = pyomo.Objective(rule = objective_func, sense = pyomo.maximize)
        # constraints 
        model.requestConstraint = pyomo.Constraint(rule=request)
        model.carbontargetConstraint = pyomo.Constraint(rule=carbon_target)
        model.demandConstraint = pyomo.Constraint(rule=demand)
        model.generatorConstraint = pyomo.Constraint(rule=generator_capacity)

        # solve 
        # objective 
        print(model.objective)
        result = solver.solve(model)
        print(result.solver.status)
        print(result.solver.termination_condition)
        print(model.display())
    return model

In [37]:
buyer_model(bdf)
seller_model(sdf)

objective
ok
optimal
Model unknown

  Variables:
    g_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :  None : False : False : NonNegativeReals
    h_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :  None : False : False : NonNegativeReals
    QS_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    QF_i : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    Y_it : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.1

  Cons

ValueError: Model objective (objective) contains nonlinear terms that cannot be written to LP format